In [1]:
library(tidyverse)
library(tmlenet)
library(arrow)
library(igraph)
library(Matrix)
library(simcausal)

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
tmlenet

The tmlenet package is still in beta testing. Interpret results with caution.


Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp



Attaching package: ‘igraph’


The following objects are masked from ‘package:lubridate’:

    %--%, union


The following objects are masked from ‘package:dplyr’:

    as_d

In [2]:
datO.feature <- read_feather(
    paste0('data/feature_', formatC(0, flag=0, width=3),'.feather')
)
datO.network <- read_feather('data/network.feather')

In [3]:
datO.feature$T.new1 = 1
datO.feature$T.new2 = 0

In [4]:
n_node = dim(datO.feature)[1]

In [5]:
datO.feature

X1,X2,X3,W1,W2,W3,p,T,m,Y,T.new1,T.new2
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>
0,1,1,-0.25,0.25,-0.25,0.5000000,1,0.62245933,0,1,0
0,0,1,0.25,-0.25,-0.25,0.2227001,1,0.90465054,1,1,0
1,0,0,-0.25,0.25,-0.25,0.5445246,0,0.32082130,1,1,0
0,1,1,-0.25,0.25,-0.25,0.6513549,1,0.40733340,1,1,0
1,1,1,0.25,0.25,0.25,0.5000000,1,0.62245933,1,1,0
0,1,0,-0.25,-0.25,0.25,0.7772999,1,0.22270014,1,1,0
0,1,0,-0.25,-0.25,0.25,0.5000000,0,0.37754067,1,1,0
1,1,1,0.25,0.25,0.25,0.6026853,1,0.47917871,0,1,0
0,0,1,0.25,-0.25,-0.25,0.7772999,0,0.09534946,0,1,0


In [6]:
G = graph.data.frame(datO.network[c(2,1)], vertices=1:n_node)

In [7]:
G_NetInd = sparseAdjMat.to.NetInd(get.adjacency(G))

In [8]:
Kmax = G_NetInd$Kmax

In [9]:
sW <- def_sW(
    X1, netX1 = mean(X1[[1:Kmax]]), X2, netX2 = mean(X2[[1:Kmax]]),
    replaceNAw0=TRUE
)

Some summary measures were not named, automatic column name(s) will be generated during evaluation



In [10]:
sA <- def_sA(T, netT = mean(T[[1:Kmax]] - 0.5), replaceNAw0=TRUE)

Some summary measures were not named, automatic column name(s) will be generated during evaluation



In [11]:
new.sA1 = def_new_sA(T = T.new1)

In [12]:
new.sA2 = def_new_sA(T = T.new2)

In [13]:
options(warn=-1)

In [14]:
iter_time = Sys.time()

res_tmlenet = tmlenet(
    data = datO.feature, NETIDmat = G_NetInd$NetInd_k, Kmax = Kmax,
    sW = sW, sA = sA,
    Ynode = "m", intervene1.sA = new.sA1, intervene2.sA = new.sA2)

message(
    paste0("elapsed time: ", 
           difftime(Sys.time(), iter_time, units="secs")[[1]], " secs.")
)

[1] "updated expression list for new.sA: "
$T
[1] "T.new1"

$netT
[1] "mean(T[[1:Kmax]] - 0.5)"

[1] "detected Anodes based on new.sA:"
[1] "T"
[1] "updated expression list for new.sA: "
$T
[1] "T.new2"

$netT
[1] "mean(T[[1:Kmax]] - 0.5)"

[1] "detected Anodes based on new.sA:"
[1] "T"


elapsed time: 2.89707493782043 secs.



In [15]:
Yh1 = res_tmlenet$EY_gstar1$estimates[1]

In [16]:
Yh2 = res_tmlenet$EY_gstar2$estimates[1]

In [17]:
Yh1

[1] 0.4908636

In [18]:
Yh2

[1] 0.5078735

In [19]:
Yh1 - Yh2

[1] -0.01700988